In [1]:
# Import packages
import laspy
import numpy as np
from numpy import ndarray
from laspy import LasData
from sklearn.cluster import KMeans

In [3]:
# Constants
SOURCE_FILEPATH = "./data/dane.las"
NEW_FILEPATH = "./result_files/{}.laz"
DIMENSIONS = ["x", "y", "z", "intensity", "red", "green", "blue"]

In [4]:
# Read las file
def read_las(filepath) -> LasData:
    las_data = laspy.read(filepath)
    return las_data

# Prepare las data
def prepare_data(las: LasData, dimensions: list) -> ndarray:
    data = None
    if len(dimensions) == 1:
        data = np.array(las[dimensions[0]])
    elif len(dimensions) > 1:        
        attributes_array = [las[dim] for dim in dimensions]    
        data = np.stack(attributes_array, -1)        
    return data

# Get dimension names
def get_dimension_names(las: LasData):
    return list(las.point_format.dimension_names)

In [10]:
def classify_point_cloud(filepath: str, dimensions: list, clusters: int):
    las = read_las(filepath)
    data = prepare_data(las, dimensions)

    kmeans = KMeans(clusters, n_init="auto")  # init="k-means++"

    new_filename = NEW_FILEPATH.format(f"kmeans_{''.join(dimensions)}_k{clusters}")
    reclassify_to_new_file(filepath, new_filename, np.array(kmeans.fit_predict(data)))
    
    
def reclassify_to_new_file(source_filepath: str, new_filepath: str, classification: ndarray):
    las = laspy.read(source_filepath)
    las.classification = classification
    las.write(new_filepath)

In [11]:
# Cluster points, reclassify and save to new laz file
classify_point_cloud(SOURCE_FILEPATH, DIMENSIONS, 10)